In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import keras
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten, AveragePooling2D, Input
from keras.optimizers import Adam, SGD
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2

np.random.seed(10)

Using TensorFlow backend.


In [0]:
# Kuzushiji MNIST
from mlxtend.data import loadlocal_mnist
X_train, y_train = loadlocal_mnist(
        images_path='/content/train-images-idx3-ubyte', 
        labels_path='/content/train-labels-idx1-ubyte')

X_test, y_test = loadlocal_mnist(
        images_path='/content/t10k-images-idx3-ubyte', 
        labels_path='/content/t10k-labels-idx1-ubyte')

In [0]:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train/=255
X_test/=255

number_of_classes = 10

Y_train = np_utils.to_categorical(y_train, number_of_classes)
Y_test = np_utils.to_categorical(y_test, number_of_classes)

In [0]:
n = 3
depth = n * 6 + 2

input_shape = X_train.shape[1:]

def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv 2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x
  
def resnet_v1(input_shape, depth, num_classes=10):
    """ResNet Version 1 Model builder [a]

    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
    Last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filters is
    doubled. Within each stage, the layers have the same number filters and the
    same number of filters.
    Features maps sizes:
    stage 0: 32x32, 16
    stage 1: 16x16, 32
    stage 2:  8x8,  64
    The Number of parameters is approx the same as Table 6 of [a]:
    ResNet20 0.27M
    ResNet32 0.46M
    ResNet44 0.66M
    ResNet56 0.85M
    ResNet110 1.7M

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    # Start model definition.
    num_filters = 16
    num_res_blocks = n

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            # commented this bcoz image is too small to downsample
            #if stack > 0 and res_block == 0:  # first layer but not first stack
                #strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [5]:
model = resnet_v1(input_shape=input_shape, depth=depth)

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(momentum=0.9),
              metrics=['accuracy'])
history = model.fit(X_train, Y_train, batch_size=128, epochs=50, verbose=1, validation_data=(X_test, Y_test))

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 51s 851us/step - loss: 0.9122 - acc: 0.7826 - val_loss: 3.3635 - val_acc: 0.2194
Epoch 2/50
60000/60000 [==============================] - 46s 763us/step - loss: 0.4069 - acc: 0.9313 - val_loss: 5.9555 - val_acc: 0.2086
Epoch 3/50
60000/60000 [==============================] - 47s 778us/step - loss: 0.3228 - acc: 0.9542 - val_loss: 3.0054 - val_acc: 0.4439
Epoch 4/50
60000/60000 [==============================] - 47s 776us/step - loss: 0.2882 - acc: 0.9625 - val_loss: 3.4767 - val_acc: 0.4015
Epoch 5/50
60000/60000 [==============================] - 47s 778us/step - loss: 0.2634 - acc: 0.9675 - val_loss: 1.1674 - val_acc: 0.6802
Epoch 6/50
60000/60000 [==============================] - 47s 780us/step - loss: 0.2460 - acc: 0.9723 - val_loss: 1.5144 - val_acc: 0.5411
Epoch 7/50
60000/60000 [==============================] - 47s 779us/

In [7]:
score = model.evaluate(X_test, Y_test)

10000/10000 [==============================] - 3s 303us/step


In [0]:
loss_history = history.history['loss']
accuracy_history = history.history['acc']
val_loss_history = history.history['val_loss']
val_accuracy_history = history.history['val_acc']

import csv

with open('resnet-sigmoid.csv', "w") as outfile:
    outfile.write("loss,accuracy,val_loss,val_acc")
    outfile.write("\n")
    for ind in range(len(loss_history)):
        outfile.write(str(loss_history[ind])+','+str(accuracy_history[ind])+','+str(val_loss_history[ind])+','+str(val_accuracy_history[ind]))
        outfile.write("\n")
        
with open('resnet-sigmoid-modelevaluate.csv', "w") as outfile:
    outfile.write("loss,")
    outfile.write("accuracy")
    outfile.write("\n")
    outfile.write(str(score[0])+',')
    outfile.write(str(score[1]))
    outfile.write("\n")